In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

In [9]:
cd ..

c:\Padhai\linkedin_fake_job_post_detector


C:\Users\GAURAV\AppData\Roaming\Python\Python313\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
from src.ScamSniffer.utils.common import read_yaml
from pathlib import Path
content = read_yaml(Path("config.yaml"))

In [12]:
data_folder = content.data_ingestion.root_dir

In [30]:
# loading data
dataset_1 = pd.read_csv(os.path.join(data_folder, "fake_job_postings.csv"))
dataset_2 = pd.read_csv(os.path.join(data_folder, "simulated_fake_job_posts.csv"))

In [15]:
dataset_1.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,8940.500000,0.042897,0.795302,0.491723,0.048434
std,5161.655742,0.202631,0.403492,0.499945,0.214688
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4470.750000,0.000000,1.000000,0.000000,0.000000
50%,8940.500000,0.000000,1.000000,0.000000,0.000000
75%,13410.250000,0.000000,1.000000,1.000000,0.000000
max,17880.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
dataset_1

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


In [22]:
dataset_1["description"].isnull().sum()

np.int64(1)

In [23]:
dataset_1["fraudulent"].isnull().sum()

np.int64(0)

In [24]:
raw_dataset_1 = dataset_1.where((pd.notnull(dataset_1)), '')

In [26]:
dataset_1_subset = raw_dataset_1[["description", "fraudulent"]]
dataset_1_subset.head()

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0


In [27]:
dataset_2.head()

,post_id,content,fraudulent
0,1,🎯 We're Hiring at Netflix!\n\nVirtual Intervie...,1
1,2,🚨 #Razorpay is Hiring – Marketing Intern!\n\nL...,1
2,3,🚨 #Axis Bank is Hiring – AI Engineer!\n\nLooki...,1
3,4,"Hello Connections,\n\nWe are #hiring for multi...",1
4,5,🚨 #Capgemini is Hiring – Marketing Intern!\n\n...,1


In [31]:
raw_job_posts = pd.concat([dataset_1_subset, dataset_2[["description", "fraudulent"]]])
raw_job_posts.head()

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0


In [32]:
raw_job_posts.tail()

,description,fraudulent
995,🚨 #HCL is Hiring – Data Analyst!\n\nLooking to...,1
996,🎯 We're Hiring at Google!\n\nVirtual Interview...,1
997,"Hii Everyone,\n\nWe are #hiring for multiple p...",1
998,🎯 We're Hiring at KPMG!\n\nVirtual Interview o...,1
999,🚀 Hiring Alert!! Great Opportunity!\n\n#Infosy...,1


In [33]:
raw_job_posts.shape

(18880, 2)

### Fake job post - 1, legit job post - 0

In [34]:
X = raw_job_posts['description']
Y = raw_job_posts['fraudulent']

### Train test split

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 3)

In [36]:
X_train.shape

(13216,)

In [38]:
X_test.shape

(5664,)

feature extraction


In [39]:
feature_extraction = TfidfVectorizer(min_df=1, stop_words = 'english', lowercase= True)

In [45]:
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

In [46]:
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

Training the model

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}
